In [1]:
#Imports Bildverarbeitung
import numpy as np 
import cv2
from matplotlib import pyplot as plt
from skimage import exposure

#Imports Neuro
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense #alle Neuronen sind miteinander verbunden
from keras.optimizers import Adam
from keras.utils import to_categorical

In [2]:
def plot(bild):
    cv2.imshow('debugging',bild)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
##Bild einlesen und ausgeben lassen

Bild_orig = cv2.imread('Bilder\sodoku3.jpg',-1)
Bild_Grau = cv2.imread('Bilder\sodoku3.jpg',0)
#(tresh, Bild_bin) = cv2.threshold(Bild_Grau, 100, 200, cv2.THRESH_BINARY)

hoehe, breite = Bild_Grau.shape[:2]

In [20]:
## Sodoku erkennen

Bild = np.array(Bild_orig) 
#plot(Bild_orig)

# Kantendetektion mit Canny
canny_output = cv2.Canny(Bild, 90, 200)
#plot(canny_output)

# Konturenfilterung anhand der berechneten Kanten
#TODO algorithmus recherchieren opencv
contours = None
contours, _ = cv2.findContours(canny_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
# cv2.RETR_TREE --> alle gefundenen Konturen werden gespeichert
# cv2.CHAIN_APPROX_SIMPLE --> nur die Punkte von den Ecken werden gespeichert (mehrere, nicht nur vier, aber an den vier Ecken)

# größtes rechteck finden
index_groesstes = 0
groesstes = cv2.contourArea(contours[index_groesstes])
for i in range(1, len(contours)):
    a = cv2.contourArea(contours[i])  # Fläche der Rechtecke itterativ bestimmen
    if a > groesstes:
        groesstes=a
        index_groesstes = i; 

# Koordinaten der Eckpunkte ermitteln quelle: https://www.geeksforgeeks.org/find-co-ordinates-of-contours-using-opencv-python/
approx = cv2.approxPolyDP(contours[index_groesstes], 0.009 * cv2.arcLength(contours[index_groesstes], True), True)
#Plot
cv2.drawContours(Bild, [approx], 0, (0, 0, 255), 5) 
plot(Bild)

# Bild perspektivisch verändern und zuschneiden quelle: https://www.pyimagesearch.com/2014/05/05/building-pokedex-python-opencv-perspective-warping-step-5-6/
points = approx.reshape(4,2)
rect = np.zeros((4, 2), dtype = "float32")

summe = points.sum(axis = 1)
rect[0] = points[np.argmin(summe)] #oben_links
rect[2] = points[np.argmax(summe)] #unten_rechts
differenz = np.diff(points, axis = 1)
rect[1] = points[np.argmin(differenz)] #oben_rechts
rect[3] = points[np.argmax(differenz)] #unten_links

maxWidth = 400
maxHeight = maxWidth

dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype = "float32")

M = cv2.getPerspectiveTransform(rect, dst)
warp = cv2.warpPerspective(Bild, M, (maxWidth, maxHeight))
warp = cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY)
warp = exposure.rescale_intensity(warp, out_range = (0, 255))

(h, w) = warp.shape
plot(warp)

In [5]:
#zahlen analysieren
#hier auch noch eventuelle rotation beachten
#sodoku algo
#zahlen ausfüllen

In [19]:
## Neuronales Netz zur Zahlenerkennung quelle: http://python-programmieren.maximilianwittmann.de/kunstliche-intelligenz-programmieren/

#Imports
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense #alle Neuronen sind miteinander verbunden
from keras.optimizers import Adam
from keras.utils import to_categorical

#70000 Datensätze aus der Keras Bib (28 x 28 Pixel)
mnist = keras.datasets.mnist
#Trennung in Trainings- und Testdaten
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train  = keras.utils.normalize(x_train, axis=1)
x_test  = keras.utils.normalize(x_test, axis=1)

# 2 Hidden Layers, je 128 Neuronen
model = Sequential()
model.add(Dense(128, activation='relu')) # Hidden Layer 1
model.add(Dense(128, activation='relu')) # Hidden Layer 2
model.add(Dense(10, activation='softmax')) # Output Layer

model.compile(
    optimizer='rmsprop' 
)

model.fit(
    x_train.reshape(60000, 784), 
    y_train, 
    batch_size=64, 
    epochs=3
)






Epoch 1/3


ValueError: in user code:

    C:\Users\Johanna\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\Johanna\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Johanna\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Johanna\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Johanna\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\U